In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix


In [9]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

class SpotifyDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class SpotifyRankPredictor(nn.Module):
    def __init__(self, num_categories):
        super(SpotifyRankPredictor, self).__init__()

        self.input_layer = nn.Linear(8, 64)

        self.block1 = nn.Sequential(
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.3)
        )
        self.block2 = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Dropout(0.3)
        )
        self.output_layer = nn.Linear(32, num_categories)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.input_layer(x)
        x = nn.ReLU()(x)

        identity = x
        x = self.block1(x) + identity

        x = self.block2(x)

        x = self.output_layer(x)
        return self.softmax(x)

def calculate_class_weights(categories):
    class_counts = np.bincount(categories)
    weights = 1 + np.log1p(np.max(class_counts) / class_counts)
    return torch.FloatTensor(weights)

def preprocess_data(df):
    feature_names = ['Danceability', 'Energy', 'Loudness', 'Speechiness',
                     'Acousticness', 'Liveness', 'Tempo', 'Duration (ms)']
    X = df[feature_names].values

    def rank_to_category(rank):
        if rank <= 30:
            return 0
        elif rank <= 50:
            return 1
        else:
            return 2

    ranks = df['Highest Charting Position'].values
    categories = np.array([rank_to_category(rank) for rank in ranks])

    # 원-핫 인코딩
    num_categories = 3
    y_encoded = np.eye(num_categories)[categories]

    return X, y_encoded, num_categories, feature_names

def augment_features(features, num_samples_needed, feature_names, noise_scales):
    augmented_data = []

    for _ in range(num_samples_needed):
        base_sample = features[np.random.randint(len(features))]
        new_features = []

        for feat_idx, feat_name in enumerate(feature_names):
            feature = base_sample[feat_idx]
            noise = np.random.normal(0, noise_scales[feat_name])
            new_value = feature + noise

            if feat_name == 'Loudness':
                new_value = np.clip(new_value, -60, 0)
            elif feat_name == 'Duration (ms)':
                new_value = max(1000, new_value)
            else:
                new_value = np.clip(new_value, 0, 1)

            new_features.append(new_value)

        augmented_data.append(new_features)

    return np.array(augmented_data)

def augment_and_balance_training_data(X, y, scaler, feature_names):
    # one-hot 인코딩된 y를 다시 카테고리로 변환
    categories = np.argmax(y, axis=1)

    # 각 카테고리별 샘플 수 계산
    category_counts = np.bincount(categories)
    print("Original category counts in train set:", category_counts)

    # 목표 샘플 수 설정 (가장 많은 클래스의 90%로 조정)
    target_samples = int(max(category_counts) * 0.9)
    print("Target samples per category:", target_samples)

    processed_features = []
    processed_categories = []

    # 스케일링된 데이터에서 표준편차 계산
    X_scaled = scaler.transform(X)
    noise_scales = {
        feat: np.std(X_scaled[:, i]) * 0.1
        for i, feat in enumerate(feature_names)
    }

    for category in range(3):
        category_mask = categories == category
        category_features = X[category_mask]
        current_samples = len(category_features)

        if current_samples < target_samples:
            num_samples_needed = target_samples - current_samples
            augmented = augment_features(category_features, num_samples_needed, feature_names, noise_scales)
            processed_features.append(np.vstack([category_features, augmented]))
            processed_categories.extend([category] * target_samples)
        else:
            processed_features.append(category_features)
            processed_categories.extend([category] * current_samples)

    X_combined = np.vstack(processed_features)
    categories_combined = np.array(processed_categories)

    # 스케일링된 특성
    X_scaled = scaler.transform(X_combined)

    # 원-핫 인코딩
    y_combined = np.eye(3)[categories_combined]

    print("Final category counts in balanced train set:", np.bincount(categories_combined))

    # 클래스 가중치 계산
    class_weights = calculate_class_weights(categories_combined)

    return X_scaled, y_combined, class_weights

def train_model(model, train_loader, val_loader, criterion, optimizer,
                num_epochs=150, patience=15):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    criterion = criterion.to(device)

    best_val_loss = float('inf')
    patience_counter = 0
    train_losses = []
    val_losses = []
    best_val_accuracy = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            _, actual = torch.max(y_batch.data, 1)
            train_total += y_batch.size(0)
            train_correct += (predicted == actual).sum().item()

        train_loss /= len(train_loader)
        train_accuracy = 100 * train_correct / train_total
        train_losses.append(train_loss)

        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()

                _, predicted = torch.max(outputs.data, 1)
                _, actual = torch.max(y_batch.data, 1)
                val_total += y_batch.size(0)
                val_correct += (predicted == actual).sum().item()

        val_loss /= len(val_loader)
        val_accuracy = 100 * val_correct / val_total
        val_losses.append(val_loss)

        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, '
              f'Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%')

        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            patience_counter = 0
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_accuracy': val_accuracy,
            }, 'best_model.pth')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f'Early stopping triggered! Best validation accuracy: {best_val_accuracy:.2f}%')
                break

    return train_losses, val_losses

def analyze_misclassification_patterns(model, test_loader, feature_names, scaler):
    model.eval()
    # 각 오분류 패턴별로 데이터를 저장할 딕셔너리
    misclassification_patterns = {
        (0, 1): {'features': [], 'confidences': []},  # 실제 0을 1로 예측
        (0, 2): {'features': [], 'confidences': []},  # 실제 0을 2로 예측
        (1, 0): {'features': [], 'confidences': []},  # 실제 1을 0으로 예측
        (1, 2): {'features': [], 'confidences': []},  # 실제 1을 2로 예측
        (2, 0): {'features': [], 'confidences': []},  # 실제 2를 0으로 예측
        (2, 1): {'features': [], 'confidences': []}   # 실제 2를 1로 예측
    }

    # 정확히 분류된 케이스도 저장
    correct_classifications = {
        0: {'features': [], 'confidences': []},
        1: {'features': [], 'confidences': []},
        2: {'features': [], 'confidences': []}
    }

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            probabilities = outputs  # 모델이 이미 softmax를 포함하고 있음
            _, predicted = torch.max(outputs.data, 1)
            _, actual = torch.max(y_batch.data, 1)

            # 원래 스케일로 복원
            original_features = scaler.inverse_transform(X_batch)

            for i in range(len(predicted)):
                pred = predicted[i].item()
                act = actual[i].item()
                conf = probabilities[i][pred].item()
                features = original_features[i]

                if pred == act:
                    correct_classifications[act]['features'].append(features)
                    correct_classifications[act]['confidences'].append(conf)
                else:
                    pattern = (act, pred)
                    if pattern in misclassification_patterns:
                        misclassification_patterns[pattern]['features'].append(features)
                        misclassification_patterns[pattern]['confidences'].append(conf)

    # 분석 결과를 저장할 딕셔너리
    analysis_results = {}

    # 각 오분류 패턴에 대한 통계 계산
    for pattern, data in misclassification_patterns.items():
        if len(data['features']) > 0:
            features_array = np.array(data['features'])
            mean_features = np.mean(features_array, axis=0)
            std_features = np.std(features_array, axis=0)
            mean_confidence = np.mean(data['confidences'])

            # 올바르게 분류된 케이스와의 차이 계산
            actual_class = pattern[0]
            correct_features = np.array(correct_classifications[actual_class]['features'])
            if len(correct_features) > 0:
                correct_mean = np.mean(correct_features, axis=0)
                feature_differences = mean_features - correct_mean

                analysis_results[pattern] = {
                    'count': len(data['features']),
                    'mean_confidence': mean_confidence,
                    'feature_analysis': {
                        feature: {
                            'mean': mean_features[i],
                            'std': std_features[i],
                            'diff_from_correct': feature_differences[i]
                        }
                        for i, feature in enumerate(feature_names)
                    }
                }

    return analysis_results

def print_misclassification_analysis(analysis_results, feature_names):
    class_names = ['Top 30', '31-50', '50+ or Not Charted']

    print("\n=== 오분류 패턴 분석 ===")
    for pattern, results in analysis_results.items():
        actual, predicted = pattern
        print(f"\n실제 {class_names[actual]}를 {class_names[predicted]}로 잘못 예측한 케이스:")
        print(f"발생 횟수: {results['count']}")
        print(f"평균 신뢰도: {results['mean_confidence']:.3f}")

        print("\n주요 특성 차이 (올바르게 분류된 케이스와 비교):")
        # 차이가 큰 순서대로 정렬
        sorted_features = sorted(
            feature_names,
            key=lambda x: abs(results['feature_analysis'][x]['diff_from_correct']),
            reverse=True
        )

        for feature in sorted_features[:3]:  # 상위 3개 특성만 표시
            analysis = results['feature_analysis'][feature]
            diff = analysis['diff_from_correct']
            print(f"  {feature}:")
            print(f"    평균값: {analysis['mean']:.3f} (±{analysis['std']:.3f})")
            print(f"    정상 케이스와의 차이: {diff:+.3f}")
            if abs(diff) > 0.5:  # 큰 차이가 있는 경우 추가 설명
                direction = "높음" if diff > 0 else "낮음"
                print(f"    → 잘못 분류된 케이스에서 눈에 띄게 {direction}")


def analyze_misclassifications(model, test_loader, feature_names, scaler):
    model.eval()
    misclassified_samples = []

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs.data, 1)
            _, actual = torch.max(y_batch.data, 1)

            # 잘못 예측된 샘플 찾기
            mask = (predicted != actual)
            if mask.any():
                wrong_X = X_batch[mask]
                wrong_pred = predicted[mask]
                wrong_actual = actual[mask]
                wrong_probs = outputs[mask]

                # 원래 스케일로 되돌리기
                original_features = scaler.inverse_transform(wrong_X)

                for i in range(len(wrong_X)):
                    sample = {
                        'actual': wrong_actual[i].item(),
                        'predicted': wrong_pred[i].item(),
                        'confidence': wrong_probs[i].max().item(),
                        'features': dict(zip(feature_names, original_features[i]))
                    }
                    misclassified_samples.append(sample)

    return misclassified_samples

def analyze_feature_importance(model, test_loader, feature_names):
    model.eval()
    feature_impacts = {name: [] for name in feature_names}

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            base_output = model(X_batch)

            # 각 특성에 대해 작은 변화를 주고 출력 변화 관찰
            for i, feature in enumerate(feature_names):
                X_modified = X_batch.clone()
                X_modified[:, i] += 0.1
                modified_output = model(X_modified)

                # 출력 변화량 계산
                output_change = torch.abs(modified_output - base_output).mean().item()
                feature_impacts[feature].append(output_change)

    # 평균 영향도 계산
    for feature in feature_names:
        feature_impacts[feature] = np.mean(feature_impacts[feature])

    return feature_impacts

def main():
    # 1. 데이터 로드 및 기본 전처리
    df = pd.read_csv('spotify_dataset.csv')
    X, y_encoded, num_categories, feature_names = preprocess_data(df)

    # 2. 데이터 분할 (증강 전)
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y_encoded, test_size=0.2, random_state=42,
        stratify=y_encoded.argmax(axis=1)
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, random_state=42,
        stratify=y_temp.argmax(axis=1)
    )

    # 3. StandardScaler 학습 (train set으로만)
    scaler = StandardScaler()
    scaler.fit(X_train)

    # 4. train 데이터에만 증강 및 밸런싱 적용
    X_train_processed, y_train_processed, class_weights = augment_and_balance_training_data(
        X_train, y_train, scaler, feature_names
    )

    # 5. validation과 test set은 스케일링만 적용
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)

    # 6. 데이터셋 생성
    train_dataset = SpotifyDataset(X_train_processed, y_train_processed)
    val_dataset = SpotifyDataset(X_val_scaled, y_val)
    test_dataset = SpotifyDataset(X_test_scaled, y_test)

    batch_size = 64
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    # 7. 모델, 손실함수, 옵티마이저 설정
    model = SpotifyRankPredictor(num_categories)
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=5, verbose=True
    )

    # 8. 모델 학습
    train_losses, val_losses = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        num_epochs=150,
        patience=15
    )

    # 9. 최고 성능 모델 불러오기
    checkpoint = torch.load('best_model.pth')
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    # 10. Test set evaluation
    correct = 0
    total = 0
    y_pred = []
    y_true = []

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs.data, 1)
            _, actual = torch.max(y_batch.data, 1)
            total += y_batch.size(0)
            correct += (predicted == actual).sum().item()
            y_pred.extend(predicted.numpy())
            y_true.extend(actual.numpy())

    # 11. 성능 평가 및 분석
    # 11-1. 기본 성능 지표
    accuracy = 100 * correct / total
    print(f'\nTest Accuracy: {accuracy:.2f}%')

    # 11-2. 혼동 행렬
    conf_matrix = confusion_matrix(y_true, y_pred)
    print("\nConfusion Matrix:")
    print(conf_matrix)
    print("Class weights:", class_weights)

    # 11-3. 오분류 패턴 상세 분석
    analysis_results = analyze_misclassification_patterns(
        model, test_loader, feature_names, scaler
    )
    print_misclassification_analysis(analysis_results, feature_names)

    # 11-4. 클래스별 오분류 분석
    misclassified = analyze_misclassifications(model, test_loader, feature_names, scaler)
    print("\n=== 클래스별 오분류 분석 ===")
    for category in range(3):
        wrong_predictions = [s for s in misclassified if s['actual'] == category]
        if wrong_predictions:
            avg_confidence = np.mean([s['confidence'] for s in wrong_predictions])
            print(f"\n클래스 {category} 분석:")
            print(f"오분류 개수: {len(wrong_predictions)}")
            print(f"평균 신뢰도: {avg_confidence:.2f}")

            # 각 특성별 평균값 출력
            print("\n특성별 평균값:")
            for feature in feature_names:
                values = [s['features'][feature] for s in wrong_predictions]
                print(f"{feature}: {np.mean(values):.2f} (±{np.std(values):.2f})")

    # 11-5. 특성 중요도 분석
    feature_importance = analyze_feature_importance(model, test_loader, feature_names)
    print("\n=== 특성 중요도 분석 ===")
    sorted_features = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)
    for feature, importance in sorted_features:
        print(f"{feature}: {importance:.4f}")

if __name__ == "__main__":
    main()

Original category counts in train set: [ 462  188 1102]
Target samples per category: 991
Final category counts in balanced train set: [ 991  991 1102]
Epoch [1/150], Train Loss: 1.7274, Train Acc: 53.73%, Val Loss: 1.6227, Val Acc: 63.01%


C:\anaconda3\envs\DeepL_yonsei\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [2/150], Train Loss: 1.6345, Train Acc: 60.18%, Val Loss: 1.5900, Val Acc: 63.01%
Epoch [3/150], Train Loss: 1.6214, Train Acc: 61.06%, Val Loss: 1.5860, Val Acc: 63.01%
Epoch [4/150], Train Loss: 1.6059, Train Acc: 61.93%, Val Loss: 1.5817, Val Acc: 63.01%
Epoch [5/150], Train Loss: 1.6025, Train Acc: 61.96%, Val Loss: 1.5777, Val Acc: 63.01%
Epoch [6/150], Train Loss: 1.5909, Train Acc: 62.19%, Val Loss: 1.5763, Val Acc: 63.01%
Epoch [7/150], Train Loss: 1.5893, Train Acc: 62.68%, Val Loss: 1.5762, Val Acc: 63.01%
Epoch [8/150], Train Loss: 1.5971, Train Acc: 61.93%, Val Loss: 1.5775, Val Acc: 63.01%
Epoch [9/150], Train Loss: 1.5912, Train Acc: 62.29%, Val Loss: 1.5777, Val Acc: 63.01%
Epoch [10/150], Train Loss: 1.5867, Train Acc: 62.68%, Val Loss: 1.5772, Val Acc: 63.01%
Epoch [11/150], Train Loss: 1.5814, Train Acc: 63.91%, Val Loss: 1.5766, Val Acc: 63.01%
Epoch [12/150], Train Loss: 1.5623, Train Acc: 64.07%, Val Loss: 1.5808, Val Acc: 63.01%
Epoch [13/150], Train Loss: 1

C:\Users\이강민\AppData\Local\Temp\ipykernel_3104\1658678742.py:461: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('best_model.pth')
